In [ ]:
import requests
from bs4 import BeautifulSoup
import json

Парсинг ссылок на все статьи с балансировкой L, C, R на allsides.com

In [ ]:
num_pages = 173
counter = 0
for i in range (152, num_pages):
  url = f'https://www.allsides.com/headline-roundups?page={i}'
  response = requests.get(url)

  soup = BeautifulSoup(response.content, 'html.parser')

  articles = []
  news_list = soup.find_all('td', class_='views-field views-field-name')
  for news in news_list:

      ref = news.find('a').get('href')


      article = {
          'heading': 'https://allsides.com' + ref,
      }
      articles.append(article)
      counter += 1

  with open(f"art_link/{i}page.json", "w") as f:
    json.dump(articles, f, indent=2)

print('Всего статей')
print(counter)


Функция для парсинга одной статьи в нужном json формате

In [ ]:
from requests.exceptions import ChunkedEncodingError
import random

def art_parc(url_art, label_art, counter, label_num):
  url = url_art
  label = label_art
  articles = []

  while True:
      try:
          response = requests.get(url)
          response.raise_for_status()
          break
      except ChunkedEncodingError:
          print('ChunkedEncodingError occurred. Retrying...')

  soup = BeautifulSoup(response.content, 'html.parser')
  news_left_list = soup.find_all('div', class_=f'news-item {label}')
  for news in news_left_list:
      if news.find('a', class_='news-title') is None:
          print('Skipping this news item.')
          continue

      title = news.find('a', class_='news-title').get_text().strip()

     # print(title)

      text = news.find('div', class_="body-contents").get_text().strip()

     # print(text)
      id = ''.join(random.choices(string.ascii_uppercase + string.digits, k=8))
      article = {
              'id': id,
              'title': title,
              'content': text,
              'label': label,
              'label_num': label_num
      }
      articles.append(article)

  return articles


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Основная часть работы - парсинг статей по лейблам

In [ ]:
def parc_diff_label(counter_to_start, label):
  counter = counter_to_start

  # Регулируем интервал парсинга
  # Чтобы сохранять данные при потере интернет соединения
  # И прочих необрабатываемых на уровне кода случаев

  for i in range (152,173):
    with open(f"art_link/{i}page.json", "r", encoding="utf-8") as f:
        data = json.load(f)

    # Создаем пустой массив для элементов
    elements = []

    # Итерируемся по данным и добавляем элементы в массив
    for item in data:
        elements.append(item["heading"])

    # Выводим массив элементов
    print(elements)
    for els in elements:
        url = els
        print(url)
        articles = art_parc(url, label, counter)
        with open(f"dataset/{counter}.json", "w", encoding="utf-8") as f:
          json.dump(articles, f, indent=2, ensure_ascii=False)
        counter += 1
        print("Страница номер " + f"{i}" + "\n")
        print(counter)

In [ ]:
# счётчик статей
counter_to_start = 24722
parc_diff_label(counter_to_start, 'right', 2)

Архивировать и скачать датасет

In [ ]:
!zip -r /content/dataset8.zip /content/dataset

In [ ]:
from google.colab import files
files.download("/content/dataset8.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>